In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -U transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 85.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [5]:
# ---- FIX for XLM-RoBERTa 404 issue ----
!pip install -q transformers==4.45.2 huggingface_hub==0.24.6

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "xlm-roberta-base"

# clear corrupted cache if needed
!rm -rf ~/.cache/huggingface/hub/models--FacebookAI--xlm-roberta-base
!rm -rf ~/.cache/huggingface/hub/models--xlm-roberta-base

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print("✅ Model and tokenizer loaded successfully!")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 89.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
diffusers 0.34.0 requires huggingface-hub>=0.27.0, but you have huggingface-hub 0.24.6 which is incompatible.
peft 0.16.0 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.24.6 which is incompatible.
gradio 5.38.1 requires huggingface-hub>=0.28.1, but you have huggingface-hub 0.24.6 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-690af018-0b29bc50648d02e87a4a7fbe;a1eaa51d-aa7b-4359-ac4e-f28f63820c4b)

Entry Not Found for url: https://huggingface.co/api/models/FacebookAI/xlm-roberta-base/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [9]:
# =============================
# ✅ Telugu Emotion Classification using XLM-RoBERTa-Base
# Combines and Shuffles Two Train Datasets + Uses GPU (T4 x2 Ready)
# =============================

!pip install -q transformers datasets scikit-learn torch accelerate

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset

# =====================================================
# Step 1: Load and Combine Datasets
# =====================================================
train_path1 = "/kaggle/input/nlpwadernew/final_train_except_no (1).csv"
train_path2 = "/kaggle/input/nlpwadernew/final_vader_filtered (1).csv"
val_path = "/kaggle/input/nlpwadernew/val.csv"
test_path = "/kaggle/input/nlpwadernew/test.csv"

train_df1 = pd.read_csv(train_path1)
train_df2 = pd.read_csv(train_path2)
val_df = pd.read_csv(val_path)
test_df = pd.read_csv(test_path)

# Keep only required columns
train_df1 = train_df1[['Sentence', 'Emotion']]
train_df2 = train_df2[['Sentence', 'Emotion']]
val_df = val_df[['Sentence', 'Emotion']]
test_df = test_df[['Sentence', 'Emotion']]

# Combine and shuffle the train data
combined_train_df = pd.concat([train_df1, train_df2], ignore_index=True)
combined_train_df = combined_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("✅ Combined train data shape:", combined_train_df.shape)
print("✅ Validation data shape:", val_df.shape)
print("✅ Test data shape:", test_df.shape)
print(combined_train_df.head())

# =====================================================
# Step 2: Label Encoding
# =====================================================
label_encoder = LabelEncoder()
combined_train_df["label"] = label_encoder.fit_transform(combined_train_df["Emotion"])
val_df["label"] = label_encoder.transform(val_df["Emotion"])
test_df["label"] = label_encoder.transform(test_df["Emotion"])

num_labels = len(label_encoder.classes_)
print("✅ Emotion labels:", label_encoder.classes_)

# =====================================================
# Step 3: Tokenization
# =====================================================
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["Sentence"], truncation=True, padding="max_length", max_length=128)

train_dataset = Dataset.from_pandas(combined_train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset = train_dataset.remove_columns(['Sentence', 'Emotion', '__index_level_0__'])
val_dataset = val_dataset.remove_columns(['Sentence', 'Emotion', '__index_level_0__'])
test_dataset = test_dataset.remove_columns(['Sentence', 'Emotion', '__index_level_0__'])

train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")

# =====================================================
# Step 4: Load Model
# =====================================================
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# =====================================================
# Step 5: Training Arguments (Optimized for GPU T4 × 2)
# =====================================================
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,               # Mixed precision for speed on T4 GPUs
    gradient_accumulation_steps=2,
    dataloader_num_workers=2,
    report_to="none",
)

# =====================================================
# Step 6: Define Evaluation Metrics
# =====================================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# =====================================================
# Step 7: Trainer Setup
# =====================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# =====================================================
# Step 8: Train Model
# =====================================================
trainer.train()

# =====================================================
# Step 9: Evaluate on Test Set
# =====================================================
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

print("\n✅ Classification Report on Test Set:")
print(classification_report(test_dataset['label'], preds, target_names=label_encoder.classes_))

cm = confusion_matrix(test_dataset['label'], preds)
print("\n✅ Confusion Matrix:\n", cm)


✅ Combined train data shape: (37795, 2)
✅ Validation data shape: (2433, 2)
✅ Test data shape: (2434, 2)
                                            Sentence Emotion
0                   రూ 1299 కేషియో మీస్మార్ట్‌బల్బ్‌   happy
1  నిన్న ఒక వ్యక్తిని అరెస్టు చేసి బెయిల్ లో విడు...     sad
2         రజనీని 100 కోట్లు చెల్లించమన్న జర్నలిస్టు.      no
3  పరీక్షలు ఉదయం 9:30 నుండి మధ్యాహ్నం 12:15 వరకూ ...      no
4  చెన్నై సూపర్ కింగ్స్ జట్టును వరుసగా మూడోసారి గ...   happy
✅ Emotion labels: ['angry' 'fear' 'happy' 'no' 'sad']


RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-690af8b3-1afca8ce55803f466175b746;d4823524-ce9f-4fcc-a1f8-1064671b9bc6)

Entry Not Found for url: https://huggingface.co/api/models/FacebookAI/xlm-roberta-base/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"